In [31]:
import codecs
import pandas as pd
import nbimporter
from MySQL_connection import *
from preprocessing import *
from clustering2_methods import *
from detect_es import *
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import math
from sklearn.externals import joblib
import matplotlib.pyplot as plt
import scipy
from scipy.cluster.hierarchy import ward, dendrogram, fcluster, to_tree
from sklearn.cluster import KMeans, AgglomerativeClustering,AffinityPropagation

In [ ]:
#Crear lista de materias
lista_materias = []
lista_carreras = []
lista_ids_tabla = []
lista_codigos = []
asignaturas = ieee_select_query("SELECT id, p, o, o_en FROM planes WHERE p = 'asignatura' OR p = 'responsable' OR p = 'codigo'")

for asig in asignaturas:
    #print (asig['p'])
    if asig['p'] == 'asignatura':
        lista_materias.append(str(asig['o_en']))
        lista_ids_tabla.append(asig['id'])
    elif asig['p'] == 'responsable':
        lista_carreras.append(asig['o'])
    elif asig['p'] == 'codigo':
        lista_codigos.append(asig['o'])

print(len (lista_codigos))
#for i in enumerate (lista_materias):
#    print (i)

In [ ]:
#Construir vectorizador, recibe como parámetro de entrada una funcion que toma los textos, extrae tokens y los lemas de cada token
TfidfVec = TfidfVectorizer( encoding='utf-8', tokenizer=LemNormalize, stop_words='english', use_idf=True, analyzer='word', ngram_range=(1,1))
#Entrenar los datos y obtener matriz TF-IDF
matriz_tfidf = TfidfVec.fit_transform(lista_materias)

In [ ]:
#print (TfidfVec.get_feature_names())
x_fit=pd.DataFrame(matriz_tfidf.toarray(), columns=TfidfVec.get_feature_names())
x_fit.to_csv("files/matrizTFIDF_english.csv")

In [ ]:
#Similitud del coseno: Sacar las distancias entre vectores de palabras, multiplicar la matriz TF-IDF por su transversa
matriz_distancias = (matriz_tfidf * matriz_tfidf.T).toarray()
#for a in matriz_distancias:
    #print (a)

In [ ]:
#Matriz de algoritmo jerarquico
linkage_matrix = ward(matriz_distancias) #define the linkage_matrix using ward clustering pre-computed distances
#print (linkage_matrix)

In [ ]:
#Ver como Ward va generando los clusters paso a paso
id_cluster = len(lista_materias)
for cluster in linkage_matrix:
    print (id_cluster, cluster)
    id_cluster +=1 

In [ ]:
#Imprimir clusters con los labels de las materias
fig, ax = plt.subplots(figsize=(20,80)) # set size
ax = dendrogram(linkage_matrix, labels=lista_materias, orientation="left", distance_sort='ascending');

plt.tick_params( 
    axis= 'x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    grid_linewidth=50)

plt.tight_layout() #show plot with tight layout
#uncomment below to save figure
plt.savefig('clusters_materias_english.png', dpi=200) #save figure as ward_clusters

In [29]:
clusters = fcluster(linkage_matrix, 0.95)
lista_c = list(clusters)
"""
for n in range(len(lista_materias)):
    print (lista_c[n], lista_materias[n])
"""

'\nfor n in range(len(lista_materias)):\n    print (lista_c[n], lista_materias[n])\n'

In [30]:
hierarchical_clustering = pd.DataFrame({ 
        'id_materia': lista_ids_tabla,
        'codigo': lista_codigos,
        'metodo': 'HIERARCHY',
        'cluster': lista_c,
        'asignatura': lista_materias,
        'titulacion': lista_carreras}, columns=['id_materia','codigo', 'metodo','cluster', 'asignatura', 'titulacion'])
hierarchical_clustering.to_csv("files/clustering_matrix_fcluster_english_95.csv")

In [ ]:
lengg = len (lista_materias)
f = codecs.open("files/clustering_titles_english.sql","wb",encoding="UTF-8")
f.write ("DROP TABLE IF EXISTS clustering_titles; " + 
"CREATE TABLE clustering_titles (" +
"id int(11) unsigned NOT NULL AUTO_INCREMENT," +
  "id_materia int(11) unsigned NOT NULL," +
  "codigo varchar(200) NOT NULL DEFAULT ''," +
  "metodo varchar(200) NOT NULL DEFAULT '', "+
  "cluster int(11) unsigned NOT NULL, "+
  "asignatura varchar(200) NOT NULL DEFAULT '', "+
  "titulacion varchar(200) NOT NULL DEFAULT '', "+
  "PRIMARY KEY (id) "+
") ENGINE=InnoDB DEFAULT CHARSET=utf8;\n") 
         
for it in range(len(lista_materias)):
    print ("Gua " + str(it) + " de " + str(lengg))
    idmateria = lista_ids_tabla[it]
    codigo = lista_codigos[it]
    cluster_h = lista_c[it]
    cluster_ap = clusters[it]
    asignatura= lista_materias[it]
    titulacion = lista_carrera[it]
    #if (cluster_h):
    f.write('INSERT into clustering_titles (id_materia, codigo, metodo, cluster, asignatura, titulacion) values (%s, %s,%s, %s, %s, %s)' % (idmateria, '"' + codigo + '"','"' + "HIERARCHY" + '"', cluster_h,'"' + asignatura + '"','"' + titulacion + '"' ) + ';\n')
    #elif (cluster_ap):
    f.write('INSERT into clustering_titles (id_materia, codigo, metodo, cluster, asignatura, titulacion) values (%s, %s,%s, %s, %s, %s)' % (idmateria, '"' + codigo + '"','"' + "AFFINITY-PROPAGATION" + '"', cluster_ap,'"' + asignatura + '"','"' + titulacion + '"' ) + ';\n')

In [ ]:
f.close()
plt.close()